**Pan-Tompkins QRS Complex Detection**

The first step in a lot of ECG processing is finding QRS complexes. The Pan-Tompkins algorithm is a series of pre-processing steps and a detection algorithm for finding these complexes. We'll can see it in action below.

# Imports

In [1]:
import glob
import os

from matplotlib import pyplot as plt
import mpld3
import numpy as np
import pandas as pd
import scipy as sp
import scipy.io
import scipy.signal

np.warnings.filterwarnings('ignore')

We will be using the ECGs in these files as examples

In [2]:
fs = 300
f0 = 'A00010.npz'
f1 = 'A01593.npz'

A simple version of the algorithm

In [ ]:
def BandpassFilter(signal, fs=300):
    """Bandpass filter the signal between 5 and 15 Hz."""
    b, a = sp.signal.butter(3, (5, 15), btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

def MovingSum(signal, fs=300):
    """Moving sum operation with window size of 150ms."""
    n_samples = int(round(fs * 0.150))
    return pd.Series(signal).rolling(n_samples, center=True).sum().values

def FindPeaks(signal, order=10):
    """A simple peak detection algorithm."""
    msk = (signal[order:-order] > signal[:-order * 2]) & (signal[order:-order] > signal[order * 2:])
    for o in range(1, order):
        msk &= (signal[order:-order] > signal[o: -order * 2 + o])
        msk &= (signal[order:-order] > signal[order * 2 - o: -o])
    return msk.nonzero()[0] + order

def ThresholdPeaks(filtered_signal, pks):
    """Threshold detected peaks to select the QRS complexes."""
    thresh = np.mean(filtered_signal[pks])
    return pks[filtered_signal[pks] > thresh]

def LocalizeMaxima(signal, pks, fs):
    """Find R-wave peaks in the original, unfiltered signal.
    
    This function is mostly for aesthetic purposes and not part of the algorithm.
    We only do it here because we know that the R-waves are pointing up.
    """
    tpks = []
    n_samples = int(round(fs * 0.150))
    for p in pks:
        tpks.append(np.argmax(signal[p - n_samples: p + n_samples]) + p - n_samples)
    return tpks

def PanTompkinsPeaks(signal, fs):
    """Pan-Tompkins QRS complex detection algorithm."""
    filtered_signal = MovingSum(
        np.square(
            np.diff(
                BandpassFilter(signal, fs))), fs)
    fiducial_marks = ThresholdPeaks(filtered_signal, FindPeaks(filtered_signal))
    return LocalizeMaxima(signal, fiducial_marks, fs)